<a href="https://colab.research.google.com/github/hamza3e/fall_det_erg/blob/main/Image_Dataset_Downloads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Downloading Datasets as Image**




In [1]:
mkdir /content/fall_image_datasets/

**le2i** from kaggle. kaggle.json file should be added to content folder

In [2]:
! pip install -q kaggle

In [3]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d 'tuyenldvn/falldataset-imvia'

100% 9.36G/9.37G [01:51<00:00, 117MB/s]
100% 9.37G/9.37G [01:51<00:00, 90.3MB/s]


In [6]:
!unzip -q falldataset-imvia.zip -d falldataset-imvia

In [7]:
!rm falldataset-imvia.zip

In [8]:
!mkdir /content/fall_image_datasets/le2i/

In [25]:
import cv2
import os
import numpy as np

def read_avi(folder, video_name, path):

    output_dir = os.path.join(folder, video_name)
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    if fps == 120: #for multicam
        fps = 30
    frame_numbers = [int(i) for i in np.linspace(0, frame_count, frame_count // fps * 10)]

    index = 0
    for i in range(0, frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        if i in frame_numbers:
            frame_name = f'{index:05d}.jpg'  # Change the image format to JPEG
            frame_path = os.path.join(output_dir, frame_name)
            cv2.imwrite(frame_path, frame, [cv2.IMWRITE_JPEG_QUALITY, 90])  # Adjust quality if using JPEG
            index += 1

    cap.release()

In [12]:
import glob
from tqdm import tqdm

img_folder = '/content/fall_image_datasets/le2i/'

avi_paths = sorted(glob.glob('/content/falldataset-imvia/**/*.avi', recursive=True))

for avi_path in tqdm(avi_paths):
    video_name = avi_path.split('/')[-3]+'_'+avi_path.split('/')[-1][:-4]
    read_avi(img_folder, video_name, avi_path)


100%|██████████| 190/190 [02:53<00:00,  1.09it/s]


In [13]:
!rm -r /content/falldataset-imvia

**Multicam Fall Detection**

In [ ]:
!wget http://www.iro.umontreal.ca/~labimage/Dataset/dataset.zip

--2023-12-05 17:49:28--  http://www.iro.umontreal.ca/~labimage/Dataset/dataset.zip
Resolving www.iro.umontreal.ca (www.iro.umontreal.ca)... 132.204.26.36
Connecting to www.iro.umontreal.ca (www.iro.umontreal.ca)|132.204.26.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3782815282 (3.5G) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]   3.52G  25.2MB/s    in 1m 48s  

2023-12-05 17:51:16 (33.5 MB/s) - ‘dataset.zip’ saved [3782815282/3782815282]



In [ ]:
!unzip -q dataset.zip

In [ ]:
!mkdir /content/fall_image_datasets/multicam/

In [29]:
# avi files in 120 fps into 30 fps

import cv2
import os
import numpy as np
import glob
from tqdm import tqdm

img_folder = '/content/fall_image_datasets/multicam/'

avi_paths = glob.glob('/content/dataset/*/*.avi')

for avi_path in tqdm(avi_paths):
    video_name = avi_path.split('/')[-2]+'_'+avi_path.split('/')[-1][:-4]
    read_avi(img_folder, video_name, avi_path)


100%|██████████| 192/192 [24:42<00:00,  7.72s/it]


In [ ]:
!rm -r /content/datasets/
!rm dataset.zip

**URFD**

In [14]:
!mkdir /content/fall_image_datasets/urfd/

In [22]:
def save_images(output_dir, image_files, fps):
    os.makedirs(output_dir, exist_ok=True)
    index = 0
    # Iterate through each frame and add frame number
    for idx, img_path in enumerate(image_files):
        if idx % int(fps/10) == 0:
            frame = cv2.imread(img_path)
            frame_name = f'{index:05d}.jpg'  # Change the image format to JPEG
            frame_path = os.path.join(output_dir, frame_name)
            cv2.imwrite(frame_path, frame, [cv2.IMWRITE_JPEG_QUALITY, 90])  # Adjust quality if using JPEG
            index += 1


In [16]:
import shutil
import os
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile
import glob
import cv2
from tqdm import tqdm

img_folder = '/content/fall_image_datasets/urfd/'
down = '/content/downloads/'
if not os.path.exists(down):
    os.makedirs(down, 0o700)

link = 'http://fenix.ur.edu.pl/~mkepski/ds/data/'

for i in tqdm(range(1,31)):
    dir_name = 'fall-'+f'{i:02d}'+'-cam0-rgb'

    with urlopen(link+dir_name+'.zip') as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall(down)

    image_files = sorted(glob.glob(down+ dir_name + '/*.png'))
    save_images(img_folder+dir_name, image_files, fps=30)

    shutil.rmtree(down+dir_name)

for i in tqdm(range(1,31)):
    dir_name = 'fall-'+f'{i:02d}'+'-cam1-rgb'

    with urlopen(link+dir_name+'.zip') as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall(down)

    image_files = sorted(glob.glob(down+ dir_name + '/*.png'))
    save_images(img_folder+dir_name, image_files, fps=30)

    shutil.rmtree(down+dir_name)

for i in tqdm(range(1,41)):
    dir_name = 'adl-'+f'{i:02d}'+'-cam0-rgb'

    with urlopen(link+dir_name+'.zip') as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall(down)

    image_files = sorted(glob.glob(down+ dir_name + '/*.png'))
    save_images(img_folder+dir_name, image_files, fps=30)

    shutil.rmtree(down+dir_name)


100%|██████████| 40/40 [05:25<00:00,  8.15s/it]


**upfall** drive folder https://drive.google.com/drive/folders/1AItqj3Ue-iv7NSdR7Qta1Ez4spRjCo58
shared folder should be added to drive.

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
!mkdir /content/fall_image_datasets/upfall/

In [28]:
import glob
import zipfile
import os
import shutil
import re
import cv2
from tqdm import tqdm

def extract_numeric_values(filename):
    # Use regular expressions to extract numeric values from the filename
    matches = re.findall(r'\d+', filename)
    return [int(match) for match in matches]

filenames = sorted(glob.glob('/content/drive/MyDrive/updf-share/UP-Fall Detection/*/*/*/*.zip'),key=extract_numeric_values)

ii=1

down = '/content/downloads/'
img_folder = '/content/fall_image_datasets/upfall/'
if not os.path.exists(down):
     os.makedirs(down, 0o700)

for filename in tqdm(filenames):

    dir_name = filename.split('/')[-1][:-11]+filename.split('/')[-1][-5:-4]
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        os.mkdir(down + dir_name)
        zip_ref.extractall(down + dir_name)
    if ii==388:
        os.remove('/content/downloads/Subject6Activity10Trial22/2018-07-06T12_03_04.483526.png') #this image is empty

    image_files = sorted(glob.glob(down + dir_name + '/*.png'))

    save_images(img_folder+dir_name, image_files, fps=20)
    shutil.rmtree(down+dir_name)
    ii+=1


100%|██████████| 1118/1118 [3:38:12<00:00, 11.71s/it]


In [ ]:
# # if you want to save these files on your drive, you can simply move them as below.
# # be careful, it may not fit into your storage due to its size.
# !mv /content/fall_image_datasets /content/drive/MyDrive/